In [ ]:
!pip -q install ipywidgets

import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, IntSlider, Button, VBox, HBox, Output
from IPython.display import display, Markdown

from src.recommender import load_shows, Preferences, recommend

shows = load_shows("data/shows.json")
print("Shows loaded:", len(shows))


In [ ]:
mood_dd = Dropdown(
    options=["comfort","light","thoughtful","dark","intense","romantic","calm","uplifting","awe","nostalgic"],
    value="comfort",
    description="Mood:"
)

genre_dd = Dropdown(
    options=["comedy","drama","thriller","crime","sci_fi","romance","documentary","reality","history"],
    value="comedy",
    description="Genre:"
)

time_sl = IntSlider(value=45, min=15, max=120, step=5, description="Time (min):")
pacing_sl = IntSlider(value=2, min=1, max=4, step=1, description="Pacing:")
intensity_sl = IntSlider(value=2, min=1, max=4, step=1, description="Intensity:")
binge_sl = IntSlider(value=4, min=1, max=5, step=1, description="Binge:")

btn = Button(description="Recommend", button_style="success")
out = Output()

display(VBox([
    Markdown("## Netflix: What Should You Watch?"),
    HBox([mood_dd, genre_dd]),
    time_sl,
    HBox([pacing_sl, intensity_sl, binge_sl]),
    btn,
    out
]))


In [ ]:
def on_click(_):
    out.clear_output()

    pref = Preferences(
        mood=mood_dd.value,
        genre=genre_dd.value,
        time_available=int(time_sl.value),
        pacing=int(pacing_sl.value),
        intensity=int(intensity_sl.value),
        binge=int(binge_sl.value)
    )

    recs = recommend(shows, pref, top_n=3)

    with out:
        display(Markdown("### Top picks for you"))
        for i, r in enumerate(recs, start=1):
            display(Markdown(
                f"**{i}. {r['title']}**  \n"
                f"- Episode length: {r['episode_minutes']} min  \n"
                f"- Score: `{r['score']}`"
            ))

            explain_df = pd.DataFrame([r["explain"]]).T.reset_index()
            explain_df.columns = ["factor", "score"]

            plt.figure()
            plt.bar(explain_df["factor"], explain_df["score"])
            plt.title(f"Why this matched: {r['title']}")
            plt.xticks(rotation=30, ha="right")
            plt.show()

btn.on_click(on_click)
